# Ejercicio de programación Regresión Lineal Multiple

In [3]:
# utilizado para manejos de directorios y rutas
import os

# Computacion vectorial y cientifica para python
import numpy as np
import pandas as pd  # ← necesario para leer CSV de Kaggle

# Librerias para graficación (trazado de gráficos)
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # Necesario para graficar superficies 3D

# llama a matplotlib a embeber graficas dentro de los cuadernillos
%matplotlib inline

## 2 Regresión lineal con multiples variables

Se implementa la regresion lineal multivariable para predecir el precio de las casas. El archivo `Datasets/ex1data2.txt` contiene un conjunto de entrenamiento de precios de casas en Portland, Oregon. La primera columna es el tamaño de la casa en metros cuadrados, la segunda columna es el numero de cuartos, y la tercera columna es el precio de la casa.

<a id="section4"></a>
### 2.1 Normalización de caracteristicas

Al visualizar los datos se puede observar que las caracteristicas tienen diferentes magnitudes, por lo cual se debe transformar cada valor en una escala de valores similares, esto con el fin de que el descenso por el gradiente pueda converger mas rapidamente.

In [5]:
# Cargar datos
data = pd.read_csv('train.csv')

# Variables predictoras: quitamos Id y SalePrice
X = data.drop(columns=["Id", "SalePrice"])

# Convertir categóricas a numéricas
X = pd.get_dummies(X, drop_first=True)

# Variable objetivo
y = data["SalePrice"].values

# Convertir X a numpy
X = X.values

m = y.size
print(m)
print("Número de características:", X.shape[1])

1460
Número de características: 244


La desviación estándar es una forma de medir cuánta variación hay en el rango de valores de una característica en particular (la mayoría de los puntos caeran en un rango de ± 2 en relación a la desviaciones estándar de la media); esta es una alternativa a tomar el rango de valores (max-min). En `numpy`, se puede usar la función `std` para calcular la desviacion estandar.

Por ejemplo, la caracteristica`X[:, 0]` contiene todos los valores de $x_1$ (tamaño de las casas) en el conjunto de entrenamiento, entonces `np.std(X[:, 0])` calcula la desviacion estandar de los tamaños de las casas.
En el momento en que se llama a la función `featureNormalize`, la columna adicional de unos correspondiente a $ x_0 = 1 $ aún no se ha agregado a $ X $.

<div class="alert alert-block alert-warning">
**Notas para la implementación:** Cuando se normalize una caracteristica, es importante almacenar los valores usados para la normalización - el valor de la media y el valor de la desviación estandar usado para los calculos. Despues de aprender los parametros del modelo, se deseara predecir los precios de casas que no se han visto antes. Dado un nuevo valor de x (area del living room y el numero de dormitorios), primero se debe normalizar x usando la media y la desviacion estandar que se empleo anteriormente en el conjunto de entrenamiento para entrenar el modelo.
</div>
<a id="featureNormalize"></a>

In [9]:
X = X.astype(float)         # <-- convierte a float
X = np.nan_to_num(X)        # <-- reemplaza NaNs con 0
def  featureNormalize(X):

    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [10]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

print(X)
print('Media calculada:', mu)
print('Desviación estandar calculada:', sigma)
print(X_norm)

[[6.000e+01 6.500e+01 8.450e+03 ... 0.000e+00 1.000e+00 0.000e+00]
 [2.000e+01 8.000e+01 9.600e+03 ... 0.000e+00 1.000e+00 0.000e+00]
 [6.000e+01 6.800e+01 1.125e+04 ... 0.000e+00 1.000e+00 0.000e+00]
 ...
 [7.000e+01 6.600e+01 9.042e+03 ... 0.000e+00 1.000e+00 0.000e+00]
 [2.000e+01 6.800e+01 9.717e+03 ... 0.000e+00 1.000e+00 0.000e+00]
 [2.000e+01 7.500e+01 9.937e+03 ... 0.000e+00 1.000e+00 0.000e+00]]
Media calculada: [5.68972603e+01 5.76232877e+01 1.05168281e+04 6.09931507e+00
 5.57534247e+00 1.97126781e+03 1.98486575e+03 1.03117123e+02
 4.43639726e+02 4.65493151e+01 5.67240411e+02 1.05742945e+03
 1.16262671e+03 3.46992466e+02 5.84452055e+00 1.51546370e+03
 4.25342466e-01 5.75342466e-02 1.56506849e+00 3.82876712e-01
 2.86643836e+00 1.04657534e+00 6.51780822e+00 6.13013699e-01
 1.86873973e+03 1.76712329e+00 4.72980137e+02 9.42445205e+01
 4.66602740e+01 2.19541096e+01 3.40958904e+00 1.50609589e+01
 2.75890411e+00 4.34890411e+01 6.32191781e+00 2.00781575e+03
 4.45205479e-02 1.09589041

Despues de `featureNormalize` la funcion es provada, se añade el temino de interseccion a `X_norm`:

In [11]:
# Añade el termino de interseccion a X
# (Columna de unos para X0)
X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)

In [12]:
print(X)

[[ 1.          0.07337496  0.2128772  ... -0.11785113  0.4676514
  -0.30599503]
 [ 1.         -0.87256276  0.64574726 ... -0.11785113  0.4676514
  -0.30599503]
 [ 1.          0.07337496  0.29945121 ... -0.11785113  0.4676514
  -0.30599503]
 ...
 [ 1.          0.30985939  0.2417352  ... -0.11785113  0.4676514
  -0.30599503]
 [ 1.         -0.87256276  0.29945121 ... -0.11785113  0.4676514
  -0.30599503]
 [ 1.         -0.87256276  0.50145724 ... -0.11785113  0.4676514
  -0.30599503]]


<a id="section5"></a>
### 2.2 Descenso por el gradiente

En el ejemplo anterior se implemento el descenso por el gradiente para un problema de regresion univariable. La unica diferencia es que ahora existe una caracteristica adicional en la matriz $X$. La función de hipótesis y la regla de actualización del descenso del gradiente por lotes permanecen sin cambios.

La implementacion de las funciones `computeCostMulti` y `gradientDescentMulti` son similares a la funcion de costo y función de descenso por el gradiente de la regresión lineal multiple es similar al de la regresion lineal multivariable. Es importante garantizar que el codigo soporte cualquier numero de caracteristicas y esten bien vectorizadas.

Se puede utilizar `shape`, propiedad de los arrays `numpy`, para identificar cuantas caracteristicas estan consideradas en el dataset.

<div class="alert alert-block alert-warning">
**Nota de implementación:** En el caso de multivariables, la función de costo puede se escrita considerando la forma vectorizada de la siguiente manera:

$$ J(\theta) = \frac{1}{2m}(X\theta - \vec{y})^T(X\theta - \vec{y}) $$

donde:

$$ X = \begin{pmatrix}
- (x^{(1)})^T - \\
- (x^{(2)})^T - \\
\vdots \\
- (x^{(m)})^T - \\ \\
\end{pmatrix} \qquad \mathbf{y} = \begin{bmatrix} y^{(1)} \\ y^{(2)} \\ \vdots \\ y^{(m)} \\\end{bmatrix}$$

La version vectorizada es eficiente cuando se trabaja con herramientas de calculo numericos computacional como `numpy`.
</div>

<a id="computeCostMulti"></a>

In [13]:
def computeCostMulti(X, y, theta):
    # Inicializa algunos valores utiles
    m = y.shape[0] # numero de ejemplos de entrenamiento

    J = 0

    # h = np.dot(X, theta)

    J = (1/(2 * m)) * np.sum(np.square(np.dot(X, theta) - y))

    return J


In [14]:
def gradientDescentMulti(X, y, theta, alpha, num_iters):

    # Inicializa algunos valores
    m = y.shape[0] # numero de ejemplos de entrenamiento

    # realiza una copia de theta, el cual será acutalizada por el descenso por el gradiente

    theta = theta.copy()

    J_history = []

    for i in range(num_iters):
        theta = theta - (alpha / m) * (np.dot(X, theta) - y).dot(X)
        J_history.append(computeCostMulti(X, y, theta))

    return theta, J_history

#### 3.2.1 Seleccionando coheficientes de aprendizaje


In [15]:
# Elegir algun valor para alpha (probar varias alternativas)
alpha = 0.01
num_iters = 400

# inicializa theta y ejecuta el descenso por el gradiente
theta = np.zeros(X.shape[1])
theta, J_history = gradientDescentMulti(X, y, theta, alpha, num_iters)

# Grafica la convergencia del costo
pyplot.plot(np.arange(len(J_history)), J_history, lw=2)
pyplot.xlabel('Numero de iteraciones')
pyplot.ylabel('Costo J')

# Muestra los resultados del descenso por el gradiente
print('theta calculado por el descenso por el gradiente (primeros 10): {:s}'.format(str(theta[:10])))

# Ejemplo de predicción (se debe normalizar igual que el entrenamiento)
# Aquí escogemos una fila del dataset original para probar
sample = data.drop(columns=["Id", "SalePrice"]).iloc[0:1]
sample = pd.get_dummies(sample, drop_first=True).reindex(columns=pd.get_dummies(data.drop(columns=["Id", "SalePrice"]), drop_first=True).columns, fill_value=0)
sample = (sample.values - mu) / sigma
sample = np.concatenate([np.ones((1, 1)), sample], axis=1)
price = np.dot(sample, theta)

print('El precio predecido para la primera casa del dataset (usando el descenso por el gradiente): ${:.0f}'.format(price[0]))


theta calculado por el descenso por el gradiente (primeros 10): [177673.560325    -3777.53684482   -545.9622917    4231.94165981
  11449.89252285   4990.28055336   2829.23156163   3077.65077473
   4365.06576528   3501.77907361]
El precio predecido para la primera casa del dataset (usando el descenso por el gradiente): $152647


<a id="section7"></a>
### 2.3 Ecuacion de la Normal

Una manera de calcular rapidamente el modelo de una regresion lineal es:

$$ \theta = \left( X^T X\right)^{-1} X^T\vec{y}$$

Utilizando esta formula no requiere que se escale ninguna caracteristica, y se obtendra una solucion exacta con un solo calculo: no hay “bucles de convergencia” como en el descenso por el gradiente.

Primero se recargan los datos para garantizar que las variables no esten modificadas. Recordar que no es necesario escalar las caracteristicas, se debe agregar la columna de unos a la matriz $X$ para tener el termino de intersección($\theta_0$).

In [19]:
def normalEqn(X, y):

    theta = np.zeros(X.shape[1])

    theta = np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)),X.T),y)

    return theta

In [20]:
theta = normalEqn(X, y);
print('Theta calculado a partir de la ecuación de la normal (primeros 10): {:s}'.format(str(theta[:10])));

# Ejemplo de predicción con la ecuación normal
price = np.dot(sample, theta)
print('Precio predecido para la primera casa del dataset (usando la ecuación de la normal): ${:.0f}'.format(price[0]))

Theta calculado a partir de la ecuación de la normal (primeros 10): [-8.83899704e+05 -2.25773313e+07  1.51547637e+05 -1.74981087e+05
 -2.29766889e+06 -2.26444234e+06 -4.93753812e+06  5.17359054e+05
  1.90659008e+05 -3.77311665e+21]
Precio predecido para la primera casa del dataset (usando la ecuación de la normal): $-582132236289915748352
